# Mandatory Challenge
## Context
You work in the data analysis team of a very important company. On Monday, the company shares some good news with you: you just got hired by a major retail company! So, let's get prepared for a huge amount of work!

Then you get to work with your team and define the following tasks to perform:   
1. You need to start your analysis using data from the past.  
2. You need to define a process that takes your daily data as an input and integrates it.  

You are in charge of the second part, so you are provided with a sample file that you will have to read daily. To complete you task, you need the following aggregates:
* One aggregate per store that adds up the rest of the values.
* One aggregate per item that adds up the rest of the values.

You can import the dataset `retail_sales` from Ironhack's database. 

## Your task
Therefore, your process will consist of the following steps:
1. Read the sample file that a daily process will save in your folder. 
2. Clean up the data.
3. Create the aggregates.
4. Write three tables in your local database: 
    - A table for the cleaned data.
    - A table for the aggregate per store.
    - A table for the aggregate per item.

## Instructions
* Read the csv you can find in Ironhack's database.
* Clean the data and create the aggregates as you consider.
* Create the tables in your local database.
* Populate them with your process.

In [1]:
import sqlalchemy
import pandas as pd
import numpy as np

In [2]:
import sqlalchemy
driver = 'mysql+pymysql'
user = 'data-students'
password = 'iR0nH@cK-D4T4B4S3'
ip = '34.65.10.136'
database = 'retail_sales'

conn_string = f'{driver}://{user}:{password}@{ip}/{database}'
conn = sqlalchemy.create_engine(conn_string)

In [3]:
raw_sales = pd.read_sql_query('SELECT * FROM retail_sales.raw_sales;', conn)

In [4]:
raw_sales.head()

,date,shop_id,item_id,item_price,item_cnt_day
0,2015-01-04,29,1469,1199.0,1.0
1,2015-01-04,28,21364,479.0,1.0
2,2015-01-04,28,21365,999.0,2.0
3,2015-01-04,28,22104,249.0,2.0
4,2015-01-04,28,22091,179.0,1.0


In [5]:
raw_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4545 entries, 0 to 4544
Data columns (total 5 columns):
date            4545 non-null datetime64[ns]
shop_id         4545 non-null int64
item_id         4545 non-null int64
item_price      4545 non-null float64
item_cnt_day    4545 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 177.6 KB


In [23]:
raw_sales.describe()

,shop_id,item_id,item_price,item_cnt_day
count,4545.000000,4545.000000,4545.000000,4545.000000
mean,34.021122,11140.459406,1031.686121,1.103630
std,16.565517,6558.649572,2073.919990,0.536967
min,2.000000,30.000000,3.000000,-1.000000
25%,22.000000,4977.000000,249.000000,1.000000
50%,31.000000,11247.000000,479.000000,1.000000
75%,50.000000,16671.000000,1192.000000,1.000000
max,59.000000,22162.000000,27990.000000,10.000000


In [ ]:
raw_sales.groupby('item_id').agg({'item_price': ['mean', 'min', 'max']})

In [54]:
#agg_per_store = raw_sales.groupby('shop_id').agg({'item_id': ['mean', 'min', 'max'], 'item_price': ['mean', 'min', 'max']})
agg_per_store = raw_sales.groupby('shop_id').agg({'item_price': ['mean', 'min', 'max', 'count', 'sum']})
agg_per_store

item_price                                 
                mean    min      max count        sum
shop_id                                              
2        1320.940000   28.0   8999.0    75   99070.50
3        2043.727273  500.0   8999.0    33   67443.00
4         752.846154   79.0   2799.0    39   29361.00
5         736.400000   99.0   3690.0    45   33138.00
6         923.428571    5.0   3999.0   126  116352.00
7         831.285714   99.0   3999.0    63   52371.00
10        841.000000    3.0   2456.0    27   22707.00
12       1473.586111   79.0   8999.0   144  212196.40
14        743.466667   58.0   3999.0    45   33456.00
15       1345.580645   49.0  19990.0    93  125139.00
16       1897.238095  249.0  14990.0    63  119526.00
18        406.793103    5.0   1649.0    87   35391.00
19        924.363636   99.0   4499.0    66   61008.00
21       1292.060345   28.0  26990.0   174  224818.50
22       1154.305556    5.0   4600.0   108  124665.00
24        825.434783    5.0   3490.0    69   56955.00
25        958.489796   28.0   4999.0   294  281796.00
26       1102.083333  149.0   5890.0   108  119025.00
27        964.962963   99.0   4499.0   162  156324.00
28        873.701493    5.0   4499.0   201  175614.00
29       1190.791667   99.0   2990.0    72   85737.00
31        777.095652   49.0   3999.0   345  268098.00
34        673.166667    5.0   2599.0    18   12117.00
35       1063.535714    5.0   3999.0    84   89337.00
37       3535.050000    5.0  27392.0    60  212103.00
38       1113.363636   58.0   4499.0    66   73482.00
39        595.470588    5.0   3999.0    51   30369.00
41       1228.000000   99.0   3499.0    30   36840.00
42       1366.100000   79.0  26990.0   240  327864.00
44       2167.727273   98.0  27990.0    66  143070.00
45        744.045455   99.0   3999.0    66   49107.00
46        970.681250   58.0   3490.0    96   93185.40
47       1027.461538  159.0   3999.0    78   80142.00
48        519.761905   49.0   2599.0    63   32745.00
49       1084.363636  198.0   3799.0    33   35784.00
50       2366.800000    5.0  26990.0    60  142008.00
51        442.142857  167.0   1189.0    21    9285.00
52        873.493600    5.0   3690.0    75   65512.02
53        692.958333    5.0   3490.0    72   49893.00
54        641.677419   58.0   3999.0   186  119352.00
55        691.548571   25.0   2599.0   105   72612.60
56        654.291667    5.0   3499.0    72   47109.00
57        709.495146   28.0  14990.0   309  219234.00
58        884.431373   28.0   3999.0   153  135318.00
59       1101.500000    5.0   6799.0   102  112353.00

In [53]:
agg_per_item = raw_sales.groupby('item_id').agg({'item_price': ['mean', 'min', 'max', 'count', 'sum'], 'shop_id': ['mean']})
agg_per_item

item_price                                      shop_id
              mean      min      max count       sum       mean
item_id                                                        
30          169.00   169.00   169.00     3    507.00  28.000000
31          363.00   363.00   363.00     3   1089.00   6.000000
32          149.00   149.00   149.00     3    447.00  31.000000
42          299.00   299.00   299.00     3    897.00  54.000000
59          249.00   249.00   249.00     3    747.00  57.000000
74          499.00   499.00   499.00     3   1497.00  25.000000
109         249.00   249.00   249.00     3    747.00  54.000000
259         249.00   249.00   249.00     3    747.00  54.000000
464         299.00   299.00   299.00     3    897.00  12.000000
482        3300.00  3300.00  3300.00     6  19800.00  37.000000
486         300.00   300.00   300.00    15   4500.00  29.200000
492         600.00   600.00   600.00     3   1800.00  55.000000
493         250.00   250.00   250.00     3    750.00  55.000000
494         380.00   380.00   380.00     3   1140.00  12.000000
686        1500.00  1500.00  1500.00     3   4500.00  12.000000
687         600.00   600.00   600.00     3   1800.00  47.000000
787         420.00   420.00   420.00     3   1260.00  22.000000
799          25.00    25.00    25.00     3     75.00  55.000000
803         100.00   100.00   100.00     3    300.00  55.000000
806         249.00   249.00   249.00     6   1494.00  25.000000
837        4600.00  4600.00  4600.00     3  13800.00  22.000000
839        3300.00  3300.00  3300.00     6  19800.00  32.000000
970         349.00   349.00   349.00     6   2094.00  44.000000
971         349.00   349.00   349.00     3   1047.00  27.000000
1007       1199.00  1199.00  1199.00     3   3597.00  15.000000
1010        499.00   499.00   499.00     3   1497.00  54.000000
1045        199.00   199.00   199.00     3    597.00  42.000000
1114        258.25   217.50   299.00     6   1549.50  25.000000
1143        399.00   399.00   399.00     3   1197.00  57.000000
1201        299.00   299.00   299.00     3    897.00   7.000000
...            ...      ...      ...   ...       ...        ...
21486       210.00   210.00   210.00     3    630.00  55.000000
21591       399.00   399.00   399.00     3   1197.00  42.000000
21605        99.00    99.00    99.00     3    297.00  27.000000
21669       149.00   149.00   149.00     3    447.00  57.000000
21674       399.00   399.00   399.00     3   1197.00   5.000000
21677        49.00    49.00    49.00     3    147.00   2.000000
21679       149.00   149.00   149.00     3    447.00  54.000000
21684       149.00   149.00   149.00     3    447.00  54.000000
21704       169.00   169.00   169.00     3    507.00  57.000000
21726        99.00    99.00    99.00     3    297.00  12.000000
21761       772.03   772.03   772.03     3   2316.09  59.000000
21762      1499.00  1499.00  1499.00     3   4497.00  42.000000
21793       999.00   999.00   999.00     3   2997.00  31.000000
21842       149.00   149.00   149.00     3    447.00  28.000000
21881       299.00   299.00   299.00     9   2691.00  42.333333
21893        28.00    28.00    28.00     3     84.00  50.000000
21902        99.00    99.00    99.00     3    297.00  15.000000
21922        99.00    99.00    99.00     6    594.00  14.500000
21962      1249.00  1249.00  1249.00     3   3747.00  46.000000
21976      1499.00  1499.00  1499.00     3   4497.00  31.000000
22058       169.00   169.00   169.00     3    507.00  54.000000
22073        99.00    99.00    99.00     3    297.00  35.000000
22076        28.00    28.00    28.00     3     84.00  57.000000
22087       119.00   119.00   119.00     9   1071.00  34.333333
22088       119.00   119.00   119.00    18   2142.00  25.833333
22091       179.00   179.00   179.00     6   1074.00  21.000000
22092       179.00   179.00   179.00     3    537.00  48.000000
22104       249.00   249.00   249.00     3    747.00  28.000000
22140       217.50   217.50   217.50    